In [1]:
import pandas as pd
import numpy as np
import neattext.functions as nfx
import string
import re
original=pd.read_excel('Copy of ge_course_2.xlsx')
def process_dataframe(file_path):
    df = pd.read_excel(file_path)
    df['Title'] = df['Title'].replace('\n', ' ', regex=True).replace('\t', ' ', regex=True)
    df['Description'] = df['Description'].replace('\n', ' ', regex=True).replace('\t', ' ', regex=True)
    df['Title'] = df['Title'].apply(lambda x: x.strip())
    df['Description'] = df['Description'].apply(lambda x: x.strip())
    df['Title'] = df['Title'].str.lower()
    df['Description'] = df['Description'].str.lower()
    # Remove punctuation, including double quotes
    punctuation_with_quotes = string.punctuation + '“”'
    translator = str.maketrans('', '', punctuation_with_quotes)
    df['Title'] = df['Title'].apply(lambda x: x.translate(translator))
    df['Description'] = df['Description'].apply(lambda x: x.translate(translator))
    # Remove non-English characters
    #df['Instructor'] = df['Instructor'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))
    df['Title'] = df['Title'].apply(nfx.remove_stopwords)
    df['Description'] = df['Description'].apply(nfx.remove_stopwords)
    print(df['Title'])
    print(df['Description'])
    return df
file_path = 'Copy of ge_course_2.xlsx'
processed_df = process_dataframe(file_path)

0                              exploration quantum world
1                                 introduction astronomy
2                      new perspectives archeology world
3                                      technology ethics
4                                    modern biology life
                             ...                        
120          man society western chinese science fiction
121    problem love western chinese poetry ancient mo...
122                   audiovisual criticism movie making
123                           music western civilization
124                                        meaning music
Name: Title, Length: 125, dtype: object
0      course designed broaden students’ perspectives...
1      ancient times human beings curious fascinated ...
2      recent decades archaeological findings shenzhe...
3      forms technology increasingly integrated lives...
4      contrast traditional biology usually studied l...
                             ...                

In [14]:
import spacy
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from spellchecker import SpellChecker

nlp = spacy.load("en_core_web_md")

def preprocess(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop])

processed_df['processed_title'] = processed_df['Title'].apply(preprocess)
processed_df['processed_description'] = processed_df['Description'].apply(preprocess)

processed_df['vector_title'] = processed_df['processed_title'].apply(lambda x: nlp(x).vector)
processed_df['vector_description'] = processed_df['processed_description'].apply(lambda x: nlp(x).vector)

vector_matrix_title = pd.DataFrame(processed_df['vector_title'].tolist())
vector_matrix_description = pd.DataFrame(processed_df['vector_description'].tolist())

def recommend_course(partial_title, num_of_rec=10):
    spell = SpellChecker()
    corrected_partial_title = " ".join([spell.correction(word) for word in partial_title.split()])

    preprocessed_partial_title = preprocess(corrected_partial_title)
    input_vector_title = nlp(preprocessed_partial_title).vector
    input_vector_description = nlp(preprocessed_partial_title).vector

    cosine_sim_titles = cosine_similarity([input_vector_title], vector_matrix_title)
    cosine_sim_descriptions = cosine_similarity([input_vector_description], vector_matrix_description)

    cosine_sim_avg = (0.5 * cosine_sim_titles) + (0.5 * cosine_sim_descriptions)

    sorted_scores = sorted(enumerate(cosine_sim_avg[0]), key=lambda x: x[1], reverse=True)
    selected_course_indices = [i[0] for i in sorted_scores[:num_of_rec]]
    selected_course_scores = [i[1] for i in sorted_scores[:num_of_rec]]

    result = processed_df['Title'].iloc[selected_course_indices]
    rec_df = pd.DataFrame(result)
    rec_df['similarity_scores'] = selected_course_scores
    rec_df['Description']=processed_df['Description']
    return rec_df

def evaluate_recommendations(partial_title, num_of_rec=10, k=5):
    recommendations = recommend_course(partial_title, num_of_rec)

    # Calculate Intra-List Similarity (ILS) for the top k recommended items
    top_k_vectors = [processed_df.loc[index, 'vector_title'] for index in recommendations.index[:k]]
    pairwise_similarities = cosine_similarity(top_k_vectors)
    ils = np.mean(pairwise_similarities[np.triu_indices(k, k=1)])

    print(f"Intra-List Similarity (ILS) for '{partial_title}' (top {k} recommendations): {ils:.2f}")

recommendation = recommend_course("science", num_of_rec=10)
print(recommendation)

# Example usage:
evaluate_recommendations("science", num_of_rec=10, k=5)

                                           Title  similarity_scores  \
28                    science technology society           0.844591   
9                     history philosophy science           0.844487   
34                    history philosophy biology           0.809825   
22                                 logic science           0.804482   
24                    understanding data science           0.803454   
38   research methods science technology studies           0.790057   
114    global genealogies chinese social science           0.784993   
6     serendipity accidental discoveries science           0.779065   
46        rethinking social science historically           0.774900   
120  man society western chinese science fiction           0.748071   

                                           Description  
28   course introduce student science technology st...  
9    course investigate fundamental development sci...  
34   course double code phi3115 course examine 

In [15]:
#ngambil averagae dari dua colum yaitu title ama description dicari cosine similarity nya
import spacy
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

nlp = spacy.load("en_core_web_md")

def preprocess(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop])

# Preprocess course titles
processed_df['processed_title'] = processed_df['Title'].apply(preprocess)
processed_df['Description'] = processed_df['Description'].apply(preprocess)

# Calculate the vector representation for each preprocessed title and description
processed_df['vector_title'] = processed_df['processed_title'].apply(lambda x: nlp(x).vector)
processed_df['vector_description'] = processed_df['Description'].apply(lambda x: nlp(x).vector)

# Convert the vectors into matrices for cosine similarity calculation
vector_matrix_title = pd.DataFrame(processed_df['vector_title'].tolist())
vector_matrix_description = pd.DataFrame(processed_df['vector_description'].tolist())

def recommend_course(partial_title, num_of_rec=10):
    # Preprocess and vectorize the input partial_title
    preprocessed_partial_title = preprocess(partial_title)
    input_vector = nlp(preprocessed_partial_title).vector

    # Compute cosine similarity between the input_vector and course title vectors (Model 1)
    cosine_sim_titles = cosine_similarity([input_vector], vector_matrix_title)

    # Compute cosine similarity between the input_vector and course description vectors (Model 2)
    cosine_sim_descriptions = cosine_similarity([input_vector], vector_matrix_description)

    # Calculate the average cosine similarity
    cosine_sim_avg = (cosine_sim_titles + cosine_sim_descriptions) / 2

    # Get the indices of recommended courses
    sorted_scores = sorted(enumerate(cosine_sim_avg[0]), key=lambda x: x[1], reverse=True)
    selected_course_indices = [i[0] for i in sorted_scores[:num_of_rec]]
    selected_course_scores = [i[1] for i in sorted_scores[:num_of_rec]]

    # Return the recommended courses along with their similarity scores
    result = processed_df['Title'].iloc[selected_course_indices]
    rec_df = pd.DataFrame(result)
    rec_df['similarity_scores'] = selected_course_scores
    rec_df['Description']=processed_df['Description']
    return rec_df

# Example usage:
recommendation = recommend_course("science", num_of_rec=10)
print(recommendation)

                                           Title  similarity_scores  \
28                    science technology society           0.844591   
9                     history philosophy science           0.844487   
34                    history philosophy biology           0.809825   
22                                 logic science           0.804482   
24                    understanding data science           0.803454   
38   research methods science technology studies           0.790057   
114    global genealogies chinese social science           0.784993   
6     serendipity accidental discoveries science           0.779065   
46        rethinking social science historically           0.774900   
120  man society western chinese science fiction           0.748071   

                                           Description  
28   course introduce student science technology st...  
9    course investigate fundamental development sci...  
34   course double code phi3115 course examine 